# 异步IO：asyncio（以下内容仅3.7+）

异步，是指在发起一个IO操作时，无需等其结束，可以同时做其他事情，在事情完成后，可得到通知。

asyncio模块包通过**async**/**await**语法实现协程代码。

asyncio是作为Python高并发网络，网络服务器，数据库连接，序列任务分发等异步框架。

异步程序要比经典的序列化程序复杂。

## **理解异步、协程**

现在我们讲了一些异步的使用，是时候解释一些概念了

**同步、异步、阻塞、非阻塞**四个词语之间的联系

- 首先要明确，前两者后后两者并不是一一对应的，它们不是在说同一件事情，但是非常类似，容易搞混
- 一般我们说异步程序是非阻塞的，而同步既有阻塞也有非阻塞的
- 非阻塞是指一个任务没做完，没有必要停在那里等它结束就可以开始下一个任务，保证一直在干活没有等待；阻塞就相反是一件事完全结束才开始另一件事
- 在非阻塞的情况下，同步与异步都有可能，它们都可以在一个任务没结束就开启下一个任务。而二者的区别在于：（且称正在进行的程序为主程序）当第一个程序做完的时候（比如网络请求终于相应了），会自动通知主程序回来继续操作第一个任务的结果，这种是异步；而同步则是需要主程序不断去问第一个程序是否已经完成。
- 四个词的区别参考[知乎回答](https://www.zhihu.com/question/19732473)

**协程与多线程的区别**

- 在非阻塞的情况下，多线程是同步的代表，协程是异步的代表。二者都开启了多个线程
- 多线程中，多个线程会竞争谁先运行，一个等待结束也不会去通知主程序，这样没有章法的随机运行会造成一些资源浪费
- 而协程中，多个线程（称为微线程）的调用和等待都是通过明确代码组织的。协程就像目标明确地执行一个又一个任务，而多线程则有一些彷徨迷茫的时间

**两种异步**

- 前面两种异步，一种是`await`只使用一个线程就可以实现任务切换，另一种是开启了多个线程，通过线程调度实现异步
- 一般只用一个线程将任务在多个函数之间来回切换，是使用yield生成器实现的，例子可以看[这篇文章最后生产消费者例子](https://www.liaoxuefeng.com/wiki/0014316089557264a6b348958f449949df42a6d3a2e542c000/001432090171191d05dae6e129940518d1d6cf6eeaaa969000)

**多进程、多线程、异步擅长方向**

- 异步和多线程都是在IO密集型任务上优势明显，因为它们的本质都是在尽量避免IO等待时间造成的资源浪费。而多进程可以利用多核优势，适合CPU密集型任务
- 相比于多线程，异步更适合每次等待时间较长、需要等待的任务较多的程序。因为多线程毕竟要创建新的线程，线程过多使线程竞争现象更加明显，资源浪费也就更多。如果每个任务等待时间过长，等待时间内势必开启了非常多任务，非常多线程，这时使用多线程就不是一个明智的决定。而异步则可以只开启一个线程在各个任务之间有条不紊进行，即能充分利用CPU资源，又不会影响程序运行效率

# 协程与任务(coroutine and task)


> 在jupyter notebook中使用asyncio.run()时发生asyncio.run() cannot be called from a running event loop的错误

> 是因为ipython中已经运行了一个event loop

> 对此，在jupyter中，将用await 直接用于演示

本处概要：最外层asyncio API接口的协程和任务使用说明。

## 协程

协程corotine可通过在对象前声明`async/await`来定义，是异步编程的首选方式。

举个如下片段的例子(Python 3.7+) ，打印 "Hello" ,等待1s，然后打印 "world"

In [9]:
import asyncio

async def main():
    print("hello")
    await asyncio.sleep(1)
    print('world')
    
#asyncio.run(main())
await main()

hello
world


如果不通过run()函数，而是直接调用main()，则不会被执行

如果要运行一个协程，asyncio提供三种机制。


- [`asyncio.run()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.run) 函数运行最顶层的入口函数，如上例举所指向“main()” 函数 

- 协程等待期间，下面的代码片段会在1s之后打印“hello”，再继续等待2s之后，会打印"world"。


In [10]:
import asyncio
import time

async def say_after(delay,what):
    print(f"{time.strftime('%X')}")
    await asyncio.sleep(delay)
    print(f"{time.strftime('%X')}:{what}")

async def main():
    print(f"started at {time.strftime('%X')}")

    await say_after(1,'hello')

    await say_after(2,'world')

    print(f"finished at {time.strftime('%X')}")

#asyncio.run(main())
await main()


started at 23:17:24
23:17:24
23:17:25:hello
23:17:25
23:17:27:world
finished at 23:17:27


- 通过[`asyncio.create_task()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.create_task) 函数封装协程作为asyncio [`Tasks`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.Task)的并发任务.

  修改以上用例，运行两次 `say_after` 实现协程并发:

In [12]:
async def main():
    task1 = asyncio.create_task(
        say_after(1, 'hello'))

    task2 = asyncio.create_task(
        say_after(2, 'world'))

    print(f"started at {time.strftime('%X')}")

    # Wait until both tasks are completed (should take
    # around 2 seconds.)
    await task1
    await task2

    print(f"finished at {time.strftime('%X')}")
#asyncio.run(main())
await main()

started at 23:20:31
23:20:31
23:20:31
23:20:32:hello
23:20:33:world
finished at 23:20:33


# 可等待对象（awaitable）

我们描述一个可等待对象是看它是否用`await`表达式修饰。很多asyncio APIs是设计为可等待对象。

有三种类型的可等待对象： `coroutines`, `Tasks` 和 `Futures`。

使用 `await` 可以将耗时等待的操作挂起，让出控制权

当协程执行的时候遇到 `await`，时间循环就会将本协程挂起，转而去执行别的协程，直到其他的协程挂起或执行完毕。

## Coroutines

Python的协程是可等待对象(awaitable)，因此也能被其他协程`coroutine`等待。

**要点**: 关于"协程"`coroutine`的术语在本文表示两种相关性紧密的概念。

- 协程函数:async def修饰的函数；

- 协程对象:通过**调用**协程函数返回的对象。

asyncio也支持传统的基于生成器的协程。

如下的coroutine对象创建了，但是前面没有await则不会被触发

In [16]:
import asyncio

async def nested():
    return 42

async def main():
    # Nothing happens if we just call "nested()".
    # A coroutine object is created but not awaited,
    # so it *won't run at all*.
    nested()

    # Let's do it differently now and await it:
    print(await nested())  # will print "42".

#asyncio.run(main())
await main()

42


## Task

它是对 coroutine 对象的进一步封装，它里面相比 coroutine 对象多了运行状态

Task项是用来调度协程并发。

当一个协程被任务函数[`asyncio.create_task()`](https://docs.python.org/3.7/library/asyncio-task.html#asyncio.create_task) 封装，那么协程将会自动进入调度运行。

In [3]:
import asyncio
 
async def execute(x):
    print('Number:', x)
    return x
 
coroutine = execute(1)
print('Coroutine:', coroutine)
print('After calling execute')
 
loop = asyncio.get_event_loop()
task = loop.create_task(coroutine)
print('Task:', task)
loop.run_until_complete(task)
print('Task:', task)
print('After calling loop')

"""
Coroutine: <coroutine object execute at 0x10e0f7830>
After calling execute
Task: <Task pending coro=<execute() running at demo.py:4>>
Number: 1
Task: <Task finished coro=<execute() done, defined at demo.py:4> result=1>
After calling loop
"""

Coroutine: <coroutine object execute at 0x00000215792D47C8>
After calling execute


F:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: coroutine 'execute' was never awaited
  import sys


RuntimeError: There is no current event loop in thread 'MainThread'.

另外定义 task 对象还有一种方式，就是直接通过 asyncio 的 ensure_future() 方法，返回结果也是 task 对象，这样的话我们就可以不借助于 loop 来定义，即使我们还没有声明 loop 也可以提前定义好 task 对象，写法如下：

In [ ]:
import asyncio

async def execute(x):
    print('Number:', x)
    return x

coroutine = execute(1)
print('Coroutine:', coroutine)
print('After calling execute')

task = asyncio.ensure_future(coroutine)
print('Task:', task)
loop = asyncio.get_event_loop()
loop.run_until_complete(task)
print('Task:', task)
print('After calling loop')

"""
Coroutine: <coroutine object execute at 0x10aa33830>
After calling execute
Task: <Task pending coro=<execute() running at demo.py:4>>
Number: 1
Task: <Task finished coro=<execute() done, defined at demo.py:4> result=1>
After calling loop
"""

## Futures

未来对象是一个特殊的最底层可等待对象，也是作为异步操作的**最终结果**

当一个`future`对象被`await`时，表示协程会持续等待，直到`future`对象的异步操作执行完毕。

`Future`对象在asyncio中需要允许基于回调代码用于`async/await`表达式。

通常而言，在应用层编程中不需要创建`Future`对象。

`Future`对象，有时候会被某些库和asyncio APIs体现是可`await`的。

In [ ]:
async def main():
    await function_that_returns_a_future_obejct()
    # this is also valid:
    await asyncio.gather(function_that_returns_a_future_obejct(),
                         some_python_coroutine())

[`loop.run_in_executor()`](https://docs.python.org/3.7/library/asyncio-eventloop.html#asyncio.loop.run_in_executor)方法是一个在底层函数返回`Future`对象的例子。

# 运行一个asyncio程序

`asyncio.run(coro,*,debug=False)`

此函数运行传入的协程，负责管理asyncio事件循环并**完结异步生成器**

此函数总是会创建一个新的事件循环并在结束时关闭之，理想情况下只应该被调用一次。


# 创建任务
## create_task(coro)

将coro协程打包为一个`TASK`排入日程准备执行。返回Task对象

该任务会在`get_running_loop()`返回的循环中执行，如果当前线程没有运行的循环规则会引发`RuntimeError`

python 3.7之前使用asyncio.ensure_future()函数

In [8]:
import asyncio
from datetime import datetime
async def coro():
    print(datetime.now().strftime("%Y%m%d%H%M%S"))
    await asyncio.sleep(1)

async def main():
    task1 = asyncio.create_task(coro())
    task2 = asyncio.ensure_future(coro())
    print(f"begin")
    await task1
    await task2
    print(f"end")
asyncio.run(main())


begin
20190328082600
20190328082600
end


# 休眠

## sleep

coroutine `asyncio.sleep(delay, result=None, *, loop=None)`

阻塞 delay 指定的秒数。

如果指定了 result，则当协程完成时将其返回给调用者。

sleep() 总是会挂起当前任务，以允许其他任务运行。

loop 参数已弃用，计划在 Python 3.10 中移除。

以下协程示例运行 5 秒，每秒显示一次当前日期:

In [ ]:
import asyncio
import datetime

async def display_date():
    loop = asyncio.get_running_loop()
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

asyncio.run(display_date())

# 并发运行任务

## gather

`awaitable asyncio.gather(*aws, loop=None, return_exceptions=False)`

并发 运行 aws 序列中的 可等待对象。

如果 aws 中的某个可等待对象为协程，它将自动作为一个任务加入日程。

如果所有可等待对象都成功完成，结果将是一个由所有返回值聚合而成的列表。结果值的顺序与 aws 中可等待对象的顺序一致。

如果 return_exceptions 为 `False` (默认)，所引发的首个异常会立即传播给等待 gather() 的任务。aws 序列中的其他可等待对象 **不会被取消** 并将继续运行。

如果 return_exceptions 为 `True`，异常会和成功的结果一样处理，并聚合至结果列表。

如果 `gather()` 被取消，所有被提交 (尚未完成) 的可等待对象也会 被取消。

如果 aws 序列中的任一 Task 或 Future 对象 被取消，它将被当作引发了 `CancelledError` 一样处理 -- 在此情况下 gather() 调用 **不会** 被取消。这是为了防止一个已提交的 Task/Future 被取消导致其他 Tasks/Future 也被取消。

In [ ]:
import asyncio

async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")

async def main():
    # Schedule three calls *concurrently*:
    await asyncio.gather(
        factorial("A", 2),
        factorial("B", 3),
        factorial("C", 4),
    )

asyncio.run(main())


# 屏蔽取消操作

awaitable asyncio.shield(aw, *, loop=None)

保护一个 [可等待对象](https://docs.python.org/zh-cn/3/library/asyncio-task.html#asyncio-awaitables) 防止其被 [`取消`](https://docs.python.org/zh-cn/3/library/asyncio-task.html#asyncio.Task.cancel)。

如果 *aw* 是一个协程，它将自动作为任务加入日程。

以下语句:

```
res = await shield(something())
```

相当于:

```
res = await something()
```

*不同之处* 在于如果包含它的协程被取消，在 `something()` 中运行的任务不会被取消。从 `something()` 的角度看来，取消操作并没有发生。然而其调用者已被取消，因此 "await" 表达式仍然会引发 [`CancelledError`](https://docs.python.org/zh-cn/3/library/asyncio-exceptions.html#asyncio.CancelledError)。

如果通过其他方式取消 `something()` (例如在其内部操作) 则 `shield()` 也会取消。

如果希望完全忽略取消操作 (不推荐) 则 `shield()` 函数需要配合一个 try/except 代码段，如下所示:

```
try:
    res = await shield(something())
except CancelledError:
    res = None
```

# 超时

coroutine asyncio.wait_for(aw, timeout, *, loop=None)

等待 aw 可等待对象 完成，指定 timeout 秒数后超时。

如果 aw 是一个协程，它将自动作为任务加入日程。

timeout 可以为 None，也可以为 float 或 int 型数值表示的等待秒数。如果 timeout 为 None，则等待直到完成。

如果发生超时，任务将取消并引发 asyncio.TimeoutError.

要避免任务 取消，可以加上 shield()。

函数将等待直到目标对象确实被取消，所以总等待时间可能超过 timeout 指定的秒数。

如果等待被取消，则 aw 指定的对象也会被取消。

loop 参数已弃用，计划在 Python 3.10 中移除




In [ ]:
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(3600)
    print('yay!')

async def main():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')

asyncio.run(main())

# 简单等待


参考 https://www.imooc.com/article/263959

https://docs.python.org/zh-cn/3/library/asyncio.html